In [2]:
import nltk

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [72]:
Text = list()
y = list()
with open('./smsspamcollection/SMSSpamCollection', 'r') as inputfile:
    for line in inputfile:
        result, text = line.split('\t')
        if result.strip() == 'ham':
            y.append(0)
        else:
            y.append(1)
            
        Text.append(text.strip())

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

In [73]:
vectorizer = CountVectorizer()
vectorizer.fit(Text)
X = vectorizer.transform(Text)

In [74]:
print cross_val_score(LogisticRegression(), X, y, cv=10, scoring=make_scorer(f1_score, average='binary', pos_label=1)).mean()

0.933348526858


In [75]:
est = LogisticRegression()
est.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

test = vectorizer.transform(test)

In [77]:
result = est.predict(test)

print ' '.join(map(str, result))

1 1 0 0 0


# Let's go with function:

In [100]:
def process(estimator, **kwargs):
    vectorizer = CountVectorizer(**kwargs)
    vectorizer.fit(Text)
    X = vectorizer.transform(Text)
    return cross_val_score(estimator(), X, y, cv=10, scoring=make_scorer(f1_score, average='binary', pos_label=1)).mean()

In [102]:
result22 = process(LogisticRegression, ngram_range=(2,2))

In [103]:
result33 = process(LogisticRegression, ngram_range=(3,3))

In [104]:
result13 = process(LogisticRegression, ngram_range=(1,3))

In [105]:
print "{:.2} {:.2} {:.2}".format(result22, result33, result13)

0.82 0.73 0.93


And now multinomialNB():

In [106]:
from sklearn.naive_bayes import MultinomialNB

In [108]:
result22_mnb = process(MultinomialNB, ngram_range=(2,2))

In [109]:
result33_mnb = process(MultinomialNB, ngram_range=(3,3))

In [110]:
result13_mnb = process(MultinomialNB, ngram_range=(1,3))

In [111]:
print "{:.2} {:.2} {:.2}".format(result22_mnb, result33_mnb, result13_mnb)

0.65 0.38 0.89


In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [115]:
vectorizer = TfidfVectorizer(ngram_range=(1,1))
vectorizer.fit(Text)
X = vectorizer.transform(Text)

In [116]:
print cross_val_score(LogisticRegression(), X, y, cv=10, scoring=make_scorer(f1_score, average='binary', pos_label=1)).mean()

0.852859955417


In [118]:
import numpy as np

In [166]:
from sklearn.model_selection import GridSearchCV

params = {'penalty' : ['l1', 'l2'], \
          'C' : np.logspace(-5, 4, 20)}

In [167]:
vectorizer = CountVectorizer(ngram_range=(1,5))
vectorizer.fit(Text)
X = vectorizer.transform(Text)

In [168]:
clf = GridSearchCV(LogisticRegression(), params, scoring=make_scorer(f1_score, average='binary', pos_label=1), cv=10)

In [169]:
clf.fit(X, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1.00000e-05,   2.97635e-05,   8.85867e-05,   2.63665e-04,
         7.84760e-04,   2.33572e-03,   6.95193e-03,   2.06914e-02,
         6.15848e-02,   1.83298e-01,   5.45559e-01,   1.62378e+00,
         4.83293e+00,   1.43845e+01,   4.28133e+01,   1.27427e+02,
         3.79269e+02,   1.12884e+03,   3.35982e+03,   1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=binary, pos_label=1),
       verbose=0)

In [170]:
print clf.best_estimator_
print clf.best_score_

LogisticRegression(C=3359.8182862837743, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
0.941616982419


Выводы:

Даже простая логистическая регрессия дает неплохое качество классификации сообщений на спам/не спам, при использовании подсчета признаков на основе ngram. 
Использование ngram при больших n не сильно осмысленно, статистики мало, но, как видно из grid_search'a C большое, поэтому переобучаться не будем.